In [22]:
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from models import new_models
from config import load_data

from ray import tune
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

import mlflow
from mlflow.tracking import MlflowClient

# Training setup

In [23]:
def fit(net, loss_function, optimizer, data_loader, num_epochs, mode, lr_scheduler, use_amp=False):
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
    for epoch in range(num_epochs):
        if epoch < num_epochs - 1:
            keys = ["train", "val"]
        else:
            keys = ["train", "val", "test"]
        for key in keys:
            dataset_size = 0
            dataset_loss = 0.0
            if key == "train":
                net.train()
            else:
                net.eval()
            for X_batch, y_batch in tqdm(data_loader[key]):
                X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
                with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
                    with torch.cuda.amp.autocast(enabled=False): # Mixed-precision support for compatible GPUs
                        batch_output = net(X_batch.float())
                        batch_loss = loss_function(batch_output, y_batch)
                    if key == "train":
                        scaler.scale(batch_loss).backward()
                        scaler.step(optimizer) 	
                        scaler.update()
                        optimizer.zero_grad()
                dataset_size += y_batch.shape[0]
                dataset_loss += y_batch.shape[0] * batch_loss.item()

            dataset_loss /= dataset_size

            # Report results to Ray Tune
            if key == "train":
                tune.report(train_loss=dataset_loss)
            elif key == "val":
                # Update learning rate
                lr_scheduler.step(metrics=dataset_loss)
                tune.report(val_loss=dataset_loss)
            else:
                tune.report(test_loss=dataset_loss)
    return net

In [24]:
from config import load_data

def train_model(config, data_dir):

    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    # Define hyperparameters
    train_size = 0.7
    val_size = 0.2
    test_size = 0.1

    sequence_length = config['sequence_length']
    batch_size = config['batch_size']
    num_epochs = config['num_epochs']
    lr = config['lr']
    weight_decay = config['weigth_decay']
    vars = config['variables']

    ld = load_data(data_dir = data_dir, target_variable = config['target_variable'])
    
    X, y = ld.create_lagged_matrix(window_size=sequence_length, vars_to_lag=vars)

    X_train, y_train, X_val, y_val, X_test, y_test = ld.split_data(X, y, train_size=train_size, val_size=val_size, test_size=test_size)

    train_dataloader = ld.create_dataloader(X_train, y_train, sequence_length, batch_size=batch_size, shuffle=True)
    val_dataloader = ld.create_dataloader(X_val, y_val, sequence_length, batch_size=batch_size, shuffle=True)
    test_dataloader = ld.create_dataloader(X_test, y_test, sequence_length, batch_size=batch_size, shuffle=False)
    
    # Model inputs
    if vars:
        input_size = len(vars) + 1
    else:
        input_size = 1
    hidden_size = config['hidden_size']
    num_layers = config['num_layers']
    output_size = 1

    if config['arch'] == "FCN":
        net = new_models.FCN(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] ==  "FCNTemporalAttention":
        net = new_models.FCNTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTM":
        net = new_models.LSTM(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMTemporalAttention":
        net = new_models.LSTMTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )
    elif config['arch'] == "LSTMSpatialTemporalAttention":
        net = new_models.LSTMSpatialTemporalAttention(input_size,
                    hidden_size,
                    num_layers,
                    output_size,
                    )

    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }
    
    net.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

    # Define your learning rate scheduler
    lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
                                           
    best_net = fit(net, loss_function, optimizer, data_loader, num_epochs, mode, lr_scheduler, use_amp=True)
    out_name = ""
    for k, v in config.items():
        if not k in ['weights_dir', 'cwd', 'variables']:
            out_name += '{}-{}_'.format(k, v)
    torch.save(best_net.state_dict(), os.path.join(config['cwd'], config['weights_dir'], out_name[:-1] + '.pth'))

# MLFlow setup

In [25]:
client = MlflowClient()
cwd = os.getcwd()
exp_base_name = "Test_of_interface"

created = 0
for i in range(100):
    try:
        exp_name = exp_base_name+"_{}".format(i)
        experiment_id = client.create_experiment(exp_name)
        created=1
        break
    except (TypeError, mlflow.exceptions.MlflowException):
        continue

if not created:
    print("ERROR: Try new experiment name.")
    sys.exit(1)

weights_root = "./model_weights/"
weights_dir = weights_root+exp_name+'/'
os.mkdir(weights_dir)

In [26]:
data_dir = "./data/"
target_variable = 'Flow_Kalltveit'

# Start experiments

In [27]:
Discharge = ['Flow_Tapping', 'Flow_Lyngsaana']
HBV = ['Mean_Flow_HBV',
 'Flow_HBV',
 'Evaporation_HBV',
 'SNOW_MELT_HBV',
 'Precipitation_HBV',
 'Groundwater_Storage_HBV',
 'Temperature_HBV',
 'Soil_Water_Storage_HBV']
Loggers = ['Water_Level_Lyngsaana',
 'Water_Temperature_Hiafossen',
 'Water_Level_Hiafossen',
 'Water_Level_Kalltveit',
 'Water_Temperature_Kalltveit_Kum',
 'Water_Temperature_Hiavatn',
 'Water_Level_Hiavatn',
 'Water_Temperature_Musdalsvatn',
 'Water_Level_Musdalsvatn',
 'Water_Temperature_Musdalsvatn_Downstream',
 'Water_Level_Musdalsvatn_Downstream',
 'Water_Temperature_Viglesdalsvatn',
 'Water_Level_Viglesdalsvatn',
 'Water_Temperature_Lyngsaana',
 'Water_Temperature_Kalltveit_River']
Loggers_1 = ['Water_Temperature_Lyngsaana', 'Water_Level_Lyngsaana']
Loggers_2 = ['Water_Temperature_Kalltveit_Kum', 'Water_Level_Kalltveit']
Loggers_3 = ['Water_Temperature_Hiavatn', 'Water_Level_Hiavatn']
Loggers_4 = ['Water_Temperature_Musdalsvatn', 'Water_Level_Musdalsvatn']
Loggers_5 = ['Water_Temperature_Musdalsvatn_Downstream',
 'Water_Level_Musdalsvatn_Downstream']
Loggers_6 = ['Water_Temperature_Viglesdalsvatn', 'Water_Level_Viglesdalsvatn']
Loggers_7 = ['Water_Temperature_Kalltveit_River']
Loggers_8 = ['Water_Level_Hiafossen']
Meto = ['Precipitation_Nilsebu',
 'Precipitation_Fister',
 'Air_Temperature_Fister',
 'Air_Temperature_Nilsebu',
 'Relative_Humidity_Nilsebu',
 'Wind_Direction_Nilsebu']
Meto_1 = ['Precipitation_Nilsebu',
 'Air_Temperature_Nilsebu',
 'Relative_Humidity_Nilsebu',
 'Wind_Direction_Nilsebu']
Meto_2 = ['Precipitation_Fister', 'Air_Temperature_Fister']
Meto_Pre_Air_Nilsebu = ['Precipitation_Nilsebu', 'Air_Temperature_Nilsebu']
Meto_Pre_Air_Fister = ['Precipitation_Fister', 'Air_Temperature_Fister']


In [28]:
from functools import partial

config = {
    "mlflow_experiment_id": experiment_id,
    "weights_dir": weights_dir,
    "cwd": cwd,
    "target_variable": target_variable,
    "arch": tune.grid_search(["LSTM", "LSTMTemporalAttention"]), # "FCN", "FCNTemporalAttention", "LSTMTemporalAttention", "LSTM", "LSTMSpatialAttention", "LSTMSpatialTemporalAttention"
    "sequence_length": tune.choice([25]),
    'num_epochs': tune.choice([150]),
    'num_layers': tune.choice([2, 3, 4]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "weigth_decay": tune.choice([0, 0.001, 0.0001]),
    "batch_size": tune.choice([128, 256, 512]),
    "hidden_size": tune.choice([32, 64]),
    "variables": tune.grid_search([
        Discharge+HBV,
        Discharge+HBV+Meto_1+Loggers_1,
        Discharge+HBV+Meto_2+Loggers_2,
    ])
}

analysis = tune.run(
    partial(train_model, data_dir=data_dir),
    config=config,
    resources_per_trial={"cpu": 12, "gpu": 1},
    num_samples=1,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name)],
)

100%|██████████| 87/87 [00:02<00:00, 36.39it/s]


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,val_loss,warmup_time
train_model_9f615_00000,2023-03-27_18-37-23,False,,723c91fe51eb4cf0a3cc05e24709ba3c,DESKTOP-D4IVECG,14,127.0.0.1,1316,7.76112,0.210976,7.76112,1679935043,0,,14,9f615_00000,36.7247,0.00390506


  0%|          | 0/87 [00:00<?, ?it/s]


(func pid=1316) Epoch 00015: reducing learning rate of group 0 to 1.9653e-04.


 20%|█▉        | 17/87 [00:00<00:00, 164.76it/s]


...

  0%|          | 0/276 [00:00<?, ?it/s]


(func pid=22192) Epoch 00014: reducing learning rate of group 0 to 1.2408e-02.


  6%|▌         | 17/276 [00:00<00:01, 165.08it/s]


(func pid=22192) Epoch 00038: reducing learning rate of group 0 to 6.2040e-03.


 94%|█████████▍| 260/276 [00:01<00:00, 130.36it/s]
2023-03-27 20:00:47,332	WARNING tune.py:691 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
100%|██████████| 79/79 [00:00<00:00, 494.00it/s]
2023-03-27 20:00:47,798	ERROR tune.py:758 -- Trials did not complete: [train_model_9f615_00016, train_model_9f615_00017, train_model_9f615_00018, train_model_9f615_00019, train_model_9f615_00023, train_model_9f615_00024, train_model_9f615_00025, train_model_9f615_00026, train_model_9f615_00027, train_model_9f615_00028, train_model_9f615_00029, train_model_9f615_00030, train_model_9f615_00031, train_model_9f615_00032, train_model_9f615_00033, train_model_9f615_00034, train_model_9f615_00035, train_model_9f615_00036, train_model_9f615_00037, train_model_9f615_00038, train_model_9f615_00039]
2023-03-27 20:00:47,799	INFO tune.py:763 -- Total run time: 5016.49 s